In [53]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import os 

In [54]:
# video metadata path
dirpath = os.getcwd()
vieo_path = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r'udder_video\video_metadata')
video_df = pd.read_csv(os.path.join(vieo_path, "video_metadata_20231117.csv"))
remove_files = ["1184_20231117_172549.bag", "1223_20231117_153008.bag", "738_20231117_104922.bag", "855_20231117_170701.bag"] # look at readdme in udder_video for more info
video_df = video_df[~video_df.filename.isin(remove_files)]

In [55]:
# VMS reoports are semicolon separates and column names have spacces and capital letters
cows_df = pd.read_csv("animal_info.csv", sep = ";")
cows_df.columns = [col.replace(" ", "_").lower() for col in cows_df.columns]
milk_df = pd.read_csv("milk_quarter.csv", sep = ";") 
milk_df.columns = [col.replace(" ", "_").lower() for col in milk_df.columns]

In [56]:
# add number of quaters with yield data 
for i in range(len(milk_df)):
    milk_qt = np.array(milk_df[["yield_lr", "yield_lf", "yield_rr", "yield_rf"]].iloc[i])
    milk_df.loc[i, "quarters_num"] = len(milk_qt[~np.isnan(milk_qt)])

# fix time and date to make them match
# I give a +-10 min margin form video to milking
video_df["start_hour0"]=[datetime.strptime(date, '%H:%M:%S').hour for date in video_df.time]
video_df["start_hour1"]=[datetime.strptime(date, '%H:%M:%S') + timedelta(minutes = 10) for date in video_df.time]
video_df["start_hour1"]=[hour.hour for hour in video_df.start_hour1]
video_df["start_hour2"]=[datetime.strptime(date, '%H:%M:%S') - timedelta(minutes = 10) for date in video_df.time]
video_df["start_hour2"]=[hour.hour for hour in video_df.start_hour2]

In [57]:
# milk report time is in AM -PM format
milk_df["start_hour"]=[datetime.strptime(str(datetime.strptime(date, '%m/%d/%Y %H:%M %p')), '%Y-%m-%d %H:%M:%S').hour for date in milk_df.begin_time]
milk_df["day"]=[datetime.strptime(str(datetime.strptime(date, '%m/%d/%Y %H:%M %p')), '%Y-%m-%d %H:%M:%S').day for date in milk_df.begin_time]
milk_df["AMPM"] = [re.findall(r"[A-Z]+", date)[0] for date in milk_df.begin_time]
for i in range(len(milk_df["start_hour"])):
    if (milk_df.loc[i, "AMPM"] == "PM") & (milk_df.loc[i, "start_hour"] < 12):
        milk_df.loc[i, "start_hour"]  =  milk_df.loc[i, "start_hour"] + 12
    elif (milk_df.loc[i, "AMPM"] == "AM") & (milk_df.loc[i, "start_hour"] ==12):
        milk_df.loc[i, "start_hour"]  =  0
        
# verify that all  milkins are from november 17 
milk_df["end_hour"]=milk_df["start_hour"]+1
milk_df = milk_df[milk_df.day ==17]
milk_df = milk_df.drop(["date","day"], axis = 1)

# Merge reports
viedo_counts = video_df[["cow", "filename"]].groupby("cow").agg(num_videos = ("filename","count")).reset_index()
# Merge reports
viedo_counts = video_df[["cow", "filename"]].groupby("cow").agg(num_videos = ("filename","count")).reset_index()
merged_cows = viedo_counts.set_index('cow').join(cows_df.set_index("animal_number"), how = "left").reset_index()
milk_videos = pd.concat([pd.merge(video_df, milk_df,  left_on=["cow", "start_hour0"], right_on=["animal_number", "start_hour"]),  pd.merge(video_df, milk_df,  left_on=["cow", "start_hour1"], right_on=["animal_number", "start_hour"]), pd.merge(video_df, milk_df,  left_on=["cow", "start_hour2"], right_on=["animal_number", "start_hour"])], axis= 0, ignore_index = True).drop_duplicates()
milk_videos = milk_videos.drop(["start_hour0", "start_hour1","start_hour2", "start_hour", "end_hour", "AMPM", "animal_number", "lactation_number"], axis = 1)
milk_videos = pd.merge(milk_videos, merged_cows,  left_on="cow", right_on = "cow")

# save merged file
milk_videos.to_csv("milk_videos.csv", index = False)